# Constraint statistics


## Preliminaries
### Import modules

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import fdrcorrection as fdr
from scipy import stats as _stats
from scipy.stats import spearmanr

sns.set_context("talk")

### Download data

In [2]:
%%bash
dx download -f -o ../data/ data/supplementary_dataset_11_full_constraint_metrics.tsv

## Load the data

In [3]:
df = pd.read_csv("../outputs/expected_variants_all_regions.tsv", sep="\t")

## Get constraint Z scores

In [4]:
def per_row_ztest(row, statistic="z"):
    if statistic == "z":
        i = 0
    elif statistic == "p":
        i = 1

    stat = proportions_ztest(
        count=row["n_obs"],
        nobs=row["n_pos"],
        value=row["prop_exp"],
        alternative="smaller",
        prop_var=row["prop_exp"],
    )[i]

    return stat

In [5]:
%%capture
df["z"] = df.apply(per_row_ztest, statistic="z", axis=1)
df["p"] = df.apply(per_row_ztest, statistic="p", axis=1)

# Print summary data
_ = df.groupby(["region", "csq"])["z"].count()
print(f"Constraint statistics by region and consequence:\n{_}")

## Get FDR-adjusted P-values
For nonsense variants only. Calculate separately for whole-transcripts and constrained regions

In [6]:
def fdr_adjustment(df, region, csq):
    """Get FDR-adjusted P-values for a given region and variant consequence."""
    # Mask regions and consequences
    m1 = df.region.isin(region)
    m2 = df.csq == csq

    # Filter the dataframe and drop cases without a P-value
    _df = df.loc[m1 & m2, ["region", "p"]].dropna().copy()

    # FDR adjustment
    _df["fdr_p"] = fdr(pvals=_df["p"])[1]

    return _df["fdr_p"]

In [7]:
# FDR adjustment is done separately for transcripts and NMD regions
# and for each distinct consequence
r1 = ["transcript"]
r2 = ["distal_nmd", "nmd_target", "long_exon"]
csq = ["synonymous", "missense", "nonsense"]

fdr_results = pd.concat([fdr_adjustment(df, region=r, csq=c) for c in csq for r in [r1, r2]])

In [8]:
# Join FDR-adjusted p-values to the original dataframe
df = df.join(fdr_results)

## Merge with gnomAD constraint data

In [9]:
gnomad = pd.read_csv(
    "../data/supplementary_dataset_11_full_constraint_metrics.tsv",
    sep="\t",
    usecols=["transcript", "pLI", "oe_lof_upper"],
).rename(columns={"transcript": "enst", "pLI": "pli", "oe_lof_upper": "loeuf"})

df = df.merge(gnomad, how="left")

## Save to output

In [10]:
df.to_csv("../outputs/expected_variants_all_regions_stats.tsv", sep="\t", index=False)

---
## Statistics

In [16]:
# Re-load the data
df = pd.read_csv("../outputs/expected_variants_all_regions_stats.tsv", sep="\t")

### Exclude regions where the synonymous Z-score is < -1

In [17]:
m1 = df.csq == "synonymous"
m2 = df.z >= -1
m = df[m1 & m2][["region", "enst"]]
df2 = df.merge(m, how="inner")

### Count regions in p-value bins

In [18]:
def get_p_stats_region(df, region, csq="nonsense", bins=[0, 0.001, 0.01, 0.05, 1]):
    m1 = df.region == region
    m2 = df.csq == csq

    df = df[m1 & m2][["n_obs", "p", "fdr_p"]]

    for p in ["p", "fdr_p"]:
        b = f"{p}_bin"
        df[b] = pd.cut(df[p], bins=bins)
        g = df.groupby(b)
        stats = g.agg({f"{p}": "count"})
        stats["none_observed"] = g["n_obs"].apply(lambda x: (x == 0).sum())
        print(f"{csq}, {region}")
        print(f"{stats}\n")

In [19]:
for r in df.region.unique():
    get_p_stats_region(df2, region=r)

nonsense, distal_nmd
                   p  none_observed
p_bin                              
(0.0, 0.001]     321             50
(0.001, 0.01]    419             88
(0.01, 0.05]    1480            694
(0.05, 1.0]    12885           2756

nonsense, distal_nmd
               fdr_p  none_observed
fdr_p_bin                          
(0.0, 0.001]     125             16
(0.001, 0.01]    205             34
(0.01, 0.05]     372             73
(0.05, 1.0]    14403           3465

nonsense, long_exon
                  p  none_observed
p_bin                             
(0.0, 0.001]    279             48
(0.001, 0.01]   221             60
(0.01, 0.05]    377            120
(0.05, 1.0]    1946            583

nonsense, long_exon
               fdr_p  none_observed
fdr_p_bin                          
(0.0, 0.001]     181             34
(0.001, 0.01]    105             14
(0.01, 0.05]     196             58
(0.05, 1.0]     2341            705

nonsense, nmd_target
                  p  none_observed


### Spearman rank Z vs LOEUF

In [20]:
m1 = df["region"] == "transcript"
m2 = df["csq"] == "nonsense"

z = df[m1 & m2]["z"]
loeuf = df[m1 & m2]["loeuf"]

print(spearmanr(z, loeuf, nan_policy="omit", alternative="two-sided"))
print("\nSee below for smallest possible P-value ('tiny')")
print(np.finfo(np.float64))

SignificanceResult(statistic=0.7833891727174719, pvalue=0.0)

See below for smallest possible P-value ('tiny')
Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
smallest_normal = 2.2250738585072014e-308   smallest_subnormal = 4.9406564584124654e-324
---------------------------------------------------------------

